In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.utils as vutils

# Ensure the Kaggle API is installed
!pip install kaggle

# Upload your Kaggle API token
from google.colab import files
files.upload()

# Download the dataset
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the fundus image dataset
!kaggle datasets download -d linchundan/fundusimage1000

# Unzip the dataset
with zipfile.ZipFile("fundusimage1000.zip", "r") as zip_ref:
    zip_ref.extractall("fundus_images")



Saving kaggle(1).json to kaggle(1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/linchundan/fundusimage1000
License(s): DbCL-1.0
100% 775M/778M [00:29<00:00, 28.6MB/s]
100% 778M/778M [00:29<00:00, 27.2MB/s]


New **Code**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.utils import save_image

import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
latent_dim = 100
img_size = 128
channels = 3
batch_size = 64
learning_rate = 0.0002
epochs = 50
sample_interval = 1000

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Dataset directory
data_dir = "/content/fundus_images/1000images"


In [ ]:
!pip install Pillow
from PIL import Image  # Import the Image class

class FundusDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name).convert('RGB') # Now Image is defined
        if self.transform:
            image = self.transform(image)
        return image

# ... rest of your code ...

# Load the dataset
dataset = FundusDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

generator = Generator().to(device)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, 3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ZeroPad2d((0, 1, 0, 1)),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Conv2d(256, 512, 3, stride=2, padding=1),
            nn.BatchNorm2d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
        )

        # The calculated flattened size is 41472
        flattened_size = 512 * 9 * 9  # Adjusted to match the debug output
        print("Calculated flattened size:", flattened_size)

        self.adv_layer = nn.Sequential(nn.Linear(flattened_size, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        print("Shape before flattening:", out.shape)
        out = out.view(out.shape[0], -1)
        print("Shape after flattening:", out.shape)
        validity = self.adv_layer(out)
        return validity

discriminator = Discriminator().to(device)



Calculated flattened size: 41472


In [ ]:
adversarial_loss = nn.BCELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))


In [ ]:
from torchvision import datasets

# Correct directory path
data_dir = "/content/fundus_images/1000images"  # Update with your correct path

# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
os.makedirs("images", exist_ok=True)
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):  # Note the use of (imgs, _) to correctly unpack the tuple
        # Rest of your training loop continues here...
        # Adversarial ground truths
        valid = torch.ones(imgs.size(0), 1, device=device, requires_grad=False)
        fake = torch.zeros(imgs.size(0), 1, device=device, requires_grad=False)

        # Configure input
        real_imgs = imgs.to(device)

        # Train Generator
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim, device=device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # Print progress
        print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

        if i % sample_interval == 0:
            save_image(gen_imgs.data[:25], f"images/{epoch}_{i}.png", nrow=5, normalize=True)



Streaming output truncated to the last 5000 lines.
Shape after flattening: torch.Size([64, 41472])
[Epoch 27/50] [Batch 21/32] [D loss: 0.5218154788017273] [G loss: 1.014983892440796]
Shape before flattening: torch.Size([64, 512, 9, 9])
Shape after flattening: torch.Size([64, 41472])
Shape before flattening: torch.Size([64, 512, 9, 9])
Shape after flattening: torch.Size([64, 41472])
Shape before flattening: torch.Size([64, 512, 9, 9])
Shape after flattening: torch.Size([64, 41472])
[Epoch 27/50] [Batch 22/32] [D loss: 0.47183337807655334] [G loss: 1.4332478046417236]
Shape before flattening: torch.Size([64, 512, 9, 9])
Shape after flattening: torch.Size([64, 41472])
Shape before flattening: torch.Size([64, 512, 9, 9])
Shape after flattening: torch.Size([64, 41472])
Shape before flattening: torch.Size([64, 512, 9, 9])
Shape after flattening: torch.Size([64, 41472])
[Epoch 27/50] [Batch 23/32] [D loss: 0.5638570189476013] [G loss: 1.2040331363677979]
Shape before flattening: torch.Size([

In [ ]:
import subprocess
subprocess.run(['cp', '-r', '/content/images', '/drive/MyDrive'])

In [ ]:
!pip install pytorch_fid
!pip install torchvision


In [ ]:
from pytorch_fid import fid_score
from torchvision.models import inception_v3
from torchvision.utils import save_image
import numpy as np

# Initialize InceptionV3 model for IS and FID calculation
inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
inception_model.eval()

# Function to calculate Inception Score
def calculate_inception_score(gen_imgs, splits=10):
    with torch.no_grad():
        pred = inception_model(gen_imgs)
        pred = torch.nn.functional.softmax(pred, dim=1).cpu().numpy()
        split_scores = []

        for k in range(splits):
            part = pred[k * (pred.shape[0] // splits): (k + 1) * (pred.shape[0] // splits), :]
            kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
            kl = np.mean(np.sum(kl, 1))
            split_scores.append(np.exp(kl))

        return np.mean(split_scores), np.std(split_scores)

# Function to calculate FID
def calculate_fid(real_imgs, gen_imgs):
    fid_value = fid_score.calculate_fid_given_paths([real_imgs, gen_imgs], batch_size=batch_size, device=device, dims=2048)
    return fid_value

# Modify the train function to include these metrics
def train(generator, discriminator, classifier, dataloader, epochs=100):
    for epoch in range(epochs):
        g_losses = []
        d_losses = []
        c_losses = []
        inception_scores = []
        fid_values = []

        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            valid = torch.ones(imgs.size(0), 1, requires_grad=False).to(device)
            fake = torch.zeros(imgs.size(0), 1, requires_grad=False).to(device)

            z = torch.randn(imgs.size(0), latent_dim).to(device)

            # Train Generator
            optimizer_G.zero_grad()
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()

            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()

            # Train Classifier
            optimizer_C.zero_grad()
            c_loss_real = classification_loss(classifier(imgs), labels)
            c_loss_fake = classification_loss(classifier(gen_imgs), labels)
            c_loss = c_loss_real + c_loss_fake
            c_loss.backward()
            optimizer_C.step()

            g_losses.append(g_loss.item())
            d_losses.append(d_loss.item())
            c_losses.append(c_loss.item())

        # Calculate metrics
        inception_score, _ = calculate_inception_score(gen_imgs)
        fid_value = calculate_fid(imgs, gen_imgs)
        inception_scores.append(inception_score)
        fid_values.append(fid_value)

        print(f"[Epoch {epoch}/{epochs}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}] [C loss: {c_loss.item()}]")
        print(f"Inception Score: {inception_score} FID: {fid_value}")

        # Save generated images at regular intervals
        if epoch % 10 == 0:
            save_image(gen_imgs.data[:25], f'images_{epoch}.png', nrow=5, normalize=True)

    return g_losses, d_losses, c_losses, inception_scores, fid_values

# Execute the training
g_losses, d_losses, c_losses, inception_scores, fid_values = train(generator, discriminator, classifier, dataloader, epochs=50)


In [ ]:
import matplotlib.pyplot as plt

# Plot the losses
plt.figure(figsize=(10, 5))
plt.plot(g_losses, label="Generator Loss")
plt.plot(d_losses, label="Discriminator Loss")
plt.plot(c_losses, label="Classifier Loss")
plt.legend()
plt.title("Losses")
plt.show()

# Plot the Inception Scores and FID values
plt.figure(figsize=(10, 5))
plt.plot(inception_scores, label="Inception Score")
plt.plot(fid_values, label="FID")
plt.legend()
plt.title("Inception Score and FID")
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Step 1: Define Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Define the architecture of the generator
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 3*256*256), # Adjust for image size 256x256
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 3, 256, 256)  # Assuming RGB images
        return img

# Step 2: Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # Define the architecture of the discriminator
        self.model = nn.Sequential(
            nn.Linear(3*256*256, 512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Step 3: Define Classifier (C) from MDPI Paper
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # Example architecture (adapt from MDPI paper)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64*64*64, 256)
        self.fc2 = nn.Linear(256, 5)  # 5 classes

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(2)(x)
        x = x.view(x.size(0), -1)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 4: Prepare Training Loop
def train(generator, discriminator, classifier, dataloader, epochs=100):
    # Loss functions
    adversarial_loss = nn.BCELoss()
    classification_loss = nn.CrossEntropyLoss()

    # Optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)
    optimizer_C = optim.Adam(classifier.parameters(), lr=0.0002)

    for epoch in range(epochs):
        for imgs, labels in dataloader:
            # Prepare labels for real and fake images
            valid = torch.ones(imgs.size(0), 1, requires_grad=False)
            fake = torch.zeros(imgs.size(0), 1, requires_grad=False)

            # Sample noise as generator input
            z = torch.randn(imgs.size(0), 100)

            # ---------------------
            #  Train Generator (G)
            # ---------------------
            optimizer_G.zero_grad()
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()

            # ---------------------
            #  Train Discriminator (D)
            # ---------------------
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()

            # ---------------------
            #  Train Classifier (C)
            # ---------------------
            optimizer_C.zero_grad()
            c_loss_real = classification_loss(classifier(imgs), labels)
            c_loss_fake = classification_loss(classifier(gen_imgs), labels)  # Assumes generator produces images with the right labels
            c_loss = c_loss_real + c_loss_fake
            c_loss.backward()
            optimizer_C.step()

        print(f"[Epoch {epoch}/{epochs}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}] [C loss: {c_loss.item()}]")

        # Save generated images at regular intervals (optional)
        if epoch % 10 == 0:
            save_image(gen_imgs.data[:25], f'images_{epoch}.png', nrow=5, normalize=True)

# Step 5: Dataset Loading and Training Execution
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load your dataset here
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize networks
generator = Generator()
discriminator = Discriminator()
classifier = Classifier()

# Train the Triple GAN
train(generator, discriminator, classifier, dataloader, epochs=100)
